In [1]:
import pickle

import numpy as np
import pandas as pd

from itertools import product, chain, combinations

from tqdm.notebook import tqdm

def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

In [2]:
df_classes = pd.read_csv('PI_classes.csv', index_col=0)

df_classes

,Bloco,Capacidade,Andar,Sala,Materia,Codigo,Turma,Horario,Oferecidas,Ocupadas
0,CTC 39 - Bloco A (etapa 1) - Sala de aula Sala...,45,ARQ05 - 1ºAndar,ARQ201,"Historia da Arte, Arquitetura e Urbanismo II",ARQ5622,02207A,2. 0910-3,25,20
1,CTC 39 - Bloco A (etapa 1) - Sala de aula Sala...,45,ARQ05 - 1ºAndar,ARQ201,"História da Arte, Arquitetura e Urbanismo III",ARQ5623,04207A,2. 1420-4,25,20
2,CTC 39 - Bloco A (etapa 1) - Sala de aula Sala...,45,ARQ05 - 1ºAndar,ARQ201,Arquitetura Latino Americana,ARQ5626,08207A,3. 0910-2,25,11
3,CTC 39 - Bloco A (etapa 1) - Sala de aula Sala...,45,ARQ05 - 1ºAndar,ARQ201,Criatividade e Inovação,EGC5027,09207,3. 1420-4,30,11
4,CTC 39 - Bloco A (etapa 1) - Sala de aula Sala...,45,ARQ05 - 1ºAndar,ARQ201,Análise Estrutural I,EMC5313,10203,4. 0730-3,20,10
...,...,...,...,...,...,...,...,...,...,...
1271,LIN Laboratório de Instrumentação Laboratório 16,16,NaN,LIN,Projeto Integrador,DAS5104,07220A,5. 1620-2,21,8
1272,LIN Laboratório de Instrumentação Laboratório 16,16,NaN,LIN,Instrumentação em Controle,DAS5151,06220B,6. 1010-2,15,10
1273,LPR Laboratório de Projetos Laboratório 16,16,NaN,LPR,Introdução à Engenharia de Controle e Automação,DAS5411,01220C,5. 1010-2,14,10
1274,LPR Laboratório de Projetos Laboratório 16,16,NaN,LPR,Introdução à Engenharia de Controle e Automação,DAS5411,01220A,5. 1330-2,14,12


In [3]:
df_rooms = pd.read_csv('PI_rooms.csv')
df_rooms

,Lugar,Sala,Carteiras,Características,Ar condicionado,Projetor,Laboratório
0,ARQ05 - 1ºAndar,ARQ201,45,"2 Ar condicionado, 1 Mesa do professor (sala d...",2,1,0
1,ARQ04 - 1ºAndar,ARQ202,40,"1 Ar condicionado, 1 Projetor multimídia, 1 Me...",1,1,0
2,ARQ03 - 1ºAndar,ARQ203,40,"2 Ar condicionado, 1 Projetor multimídia, 1 Me...",2,1,0
3,ARQ02 - Térreo,ARQ204,40,"2 Ar condicionado, 1 Projetor multimídia, 1 Qu...",2,1,0
4,ARQ09 - 2º Andar,ARQ301,20,"1 Ar condicionado, 1 Projetor multimídia, 1 Me...",1,1,0
...,...,...,...,...,...,...,...
123,NaN,PGEAS2,25,Laboratório,0,0,1
124,NaN,SPL313,30,Laboratório,0,0,1
125,NaN,SPL314,15,Laboratório,0,0,1
126,NaN,SPL315,30,Laboratório,0,0,1


# Lecture required seats fix

In [4]:
# we assume that the actual amount of seats required is at most the amount offered by the class allocated
df_classes['Ocupadas_fix'] = df_classes[['Capacidade', 'Ocupadas']].min(axis='columns')

# Lab rooms+lectures

In [5]:
labs = df_rooms[df_rooms['Laboratório'] > 0]['Sala']

# remove labs
df_rooms = df_rooms[df_rooms['Laboratório'] == 0].drop(columns=['Laboratório'])

print(f"Labs: {len(labs)}")

df_rooms

Labs: 60


,Lugar,Sala,Carteiras,Características,Ar condicionado,Projetor
0,ARQ05 - 1ºAndar,ARQ201,45,"2 Ar condicionado, 1 Mesa do professor (sala d...",2,1
1,ARQ04 - 1ºAndar,ARQ202,40,"1 Ar condicionado, 1 Projetor multimídia, 1 Me...",1,1
2,ARQ03 - 1ºAndar,ARQ203,40,"2 Ar condicionado, 1 Projetor multimídia, 1 Me...",2,1
3,ARQ02 - Térreo,ARQ204,40,"2 Ar condicionado, 1 Projetor multimídia, 1 Qu...",2,1
4,ARQ09 - 2º Andar,ARQ301,20,"1 Ar condicionado, 1 Projetor multimídia, 1 Me...",1,1
...,...,...,...,...,...,...
65,NaN,EQA018,45,"50 Carteira com cadeira separada, 1 Projetor m...",2,1
66,NaN,EQA020,35,"2 Ar condicionado, 35 Carteira com cadeira sep...",2,1
67,NaN,EQA021,35,"2 Ar condicionado, 35 Carteira com cadeira sep...",2,1
68,NaN,EQA022,35,"2 Ar condicionado, 35 Carteira com cadeira sep...",2,1


In [6]:
# remove lectures that occur in labs
df_classes = df_classes[~df_classes['Sala'].isin(labs)]
df_classes = df_classes[df_classes['Sala'] != 'LABCIG']
df_classes

,Bloco,Capacidade,Andar,Sala,Materia,Codigo,Turma,Horario,Oferecidas,Ocupadas,Ocupadas_fix
0,CTC 39 - Bloco A (etapa 1) - Sala de aula Sala...,45,ARQ05 - 1ºAndar,ARQ201,"Historia da Arte, Arquitetura e Urbanismo II",ARQ5622,02207A,2. 0910-3,25,20,20
1,CTC 39 - Bloco A (etapa 1) - Sala de aula Sala...,45,ARQ05 - 1ºAndar,ARQ201,"História da Arte, Arquitetura e Urbanismo III",ARQ5623,04207A,2. 1420-4,25,20,20
2,CTC 39 - Bloco A (etapa 1) - Sala de aula Sala...,45,ARQ05 - 1ºAndar,ARQ201,Arquitetura Latino Americana,ARQ5626,08207A,3. 0910-2,25,11,11
3,CTC 39 - Bloco A (etapa 1) - Sala de aula Sala...,45,ARQ05 - 1ºAndar,ARQ201,Criatividade e Inovação,EGC5027,09207,3. 1420-4,30,11,11
4,CTC 39 - Bloco A (etapa 1) - Sala de aula Sala...,45,ARQ05 - 1ºAndar,ARQ201,Análise Estrutural I,EMC5313,10203,4. 0730-3,20,10,10
...,...,...,...,...,...,...,...,...,...,...,...
1095,EQA023 Sala de aula Sala com carteira 45,45,NaN,EQA023,Operações Unitárias de Transferência de Calor ...,EQA5333,08216,5. 1330-2,20,20,20
1096,EQA023 Sala de aula Sala com carteira 45,45,NaN,EQA023,Fenômenos de Transferência II,EQA5416,06216,5. 1510-2,20,20,20
1097,EQA023 Sala de aula Sala com carteira 45,45,NaN,EQA023,Programação Econômica e Financeira,EPS5211,07216,6. 0730-3,31,26,26
1098,EQA023 Sala de aula Sala com carteira 45,45,NaN,EQA023,Operações Unitárias de Transferência de Quanti...,EQA5313,06215,6. 1330-2,20,15,15


# Room names

In [7]:
df_classes['Sala'].str.replace(' ','').nunique()

65

In [8]:
df_rooms['Sala'].str.replace(' ','').nunique()

68

In [9]:
df_classes['Sala'] = df_classes['Sala'].str.replace(' ', '')
df_rooms['Sala'] = df_rooms['Sala'].str.replace(' ', '')

In [10]:
# rooms in `df_classes` that are not in `df_rooms`
set(df_classes['Sala'].unique()).difference(set(df_rooms['Sala'].unique()))

set()

# Course-class matching

In [11]:
# are there multiple classes attending the same lecture at the same time in the same room?
seats_req = df_classes.groupby(['Codigo', 'Horario', 'Sala'])['Ocupadas'].sum()
seats_req

Codigo   Horario    Sala  
ARQ5115  2. 0910-3  ARQ303    27
         2. 1830-4  ARQ202    27
         7. 0730-1  ARQ311    27
ARQ5515  4. 1510-3  ARQ305    24
                    CTC207    18
                              ..
QMC5229  4. 1010-2  CTC304    37
         6. 0730-2  CTC305    37
QMC5350  2. 1510-2  EEL010    25
         6. 1010-2  EQA020    16
QMC5351  6. 1330-2  EQA020    21
Name: Ocupadas, Length: 878, dtype: int64

In [12]:
seats_avail = df_classes.groupby(['Codigo', 'Horario', 'Sala'])['Capacidade'].first()
seats_avail

Codigo   Horario    Sala  
ARQ5115  2. 0910-3  ARQ303    20
         2. 1830-4  ARQ202    40
         7. 0730-1  ARQ311    20
ARQ5515  4. 1510-3  ARQ305    40
                    CTC207    60
                              ..
QMC5229  4. 1010-2  CTC304    60
         6. 0730-2  CTC305    60
QMC5350  2. 1510-2  EEL010    40
         6. 1010-2  EQA020    35
QMC5351  6. 1330-2  EQA020    35
Name: Capacidade, Length: 878, dtype: int64

In [13]:
seats_req[seats_req > seats_avail] - seats_avail[seats_req > seats_avail]

Codigo   Horario    Sala  
ARQ5115  2. 0910-3  ARQ303     7
         7. 0730-1  ARQ311     7
ARQ5642  3. 0910-3  ARQ310    15
ARQ5658  5. 1010-2  CTC306    13
ARQ5662  5. 1010-2  ARQ309    14
ECV2101  4. 1830-3  CTC201     4
ECV5104  4. 1010-2  EQA020     2
ECV5213  5. 1010-2  ECV312     1
         6. 1010-2  ECV312     1
ECV5631  5. 0820-4  EQA016     7
ECV5643  5. 1330-2  ARQ202     2
ECV5645  5. 1510-2  ARQ204     7
ECV5648  2. 0730-3  ARQ204     5
EEL7030  3. 0820-2  CTC206    16
EGC5036  6. 1830-2  CTC102     8
EGR5611  3. 0910-3  ARQ303     1
         6. 0910-3  ARQ303     1
EGR5612  2. 1420-3  ARQ303     2
         4. 1420-3  ARQ303     2
EMC5138  6. 0820-2  CTC206     5
EMC5417  2. 1010-2  EEL010     3
         5. 0820-2  ENS211     3
EMC5443  2. 0910-3  EQA015     2
EPS7032  6. 1830-1  CTC111     2
FSC5002  3. 0730-2  EEL009     5
         4. 1330-2  EEL008     5
         4. 1620-2  CTC303     3
         5. 1620-2  ENS213     3
         6. 1330-2  EEL008     5
FSC5103  5. 1620

In [14]:
df_classes[df_classes['Codigo'] == 'EEL7030']

,Bloco,Capacidade,Andar,Sala,Materia,Codigo,Turma,Horario,Oferecidas,Ocupadas,Ocupadas_fix
232,CTC02 - Bloco B - Sala de Aula CTC 108 - Sala ...,40,Térreo,CTC108,Microprocessadores,EEL7030,04235A,2. 1510-2,18,18,18
233,CTC02 - Bloco B - Sala de Aula CTC 108 - Sala ...,40,Térreo,CTC108,Microprocessadores,EEL7030,04235B,2. 1510-2,18,16,16
467,CTC02 - Bloco B - Sala de Aula CTC 206 - Sala ...,40,1ºAndar,CTC206,Microprocessadores,EEL7030,04202A,3. 0820-2,19,19,19
468,CTC02 - Bloco B - Sala de Aula CTC 206 - Sala ...,40,1ºAndar,CTC206,Microprocessadores,EEL7030,04202B,3. 0820-2,11,11,11
469,CTC02 - Bloco B - Sala de Aula CTC 206 - Sala ...,40,1ºAndar,CTC206,Microprocessadores,EEL7030,04202C,3. 0820-2,20,20,20
470,CTC02 - Bloco B - Sala de Aula CTC 206 - Sala ...,40,1ºAndar,CTC206,Microprocessadores,EEL7030,10213,3. 0820-2,8,6,6


So there are many cases in which the rooms are overbooked. We will ignore this for now and let the solver handle it.

In [15]:
# every course-room-time combination is considered a lecture (actualy a long lecture)
df_lectures = df_classes.groupby(['Codigo', 'Horario', 'Sala'])['Ocupadas'].sum().reset_index()
df_lectures['long_lecture'] = df_lectures.index

# each row is a lecture
df_lectures

,Codigo,Horario,Sala,Ocupadas,long_lecture
0,ARQ5115,2. 0910-3,ARQ303,27,0
1,ARQ5115,2. 1830-4,ARQ202,27,1
2,ARQ5115,7. 0730-1,ARQ311,27,2
3,ARQ5515,4. 1510-3,ARQ305,24,3
4,ARQ5515,4. 1510-3,CTC207,18,4
...,...,...,...,...,...
873,QMC5229,4. 1010-2,CTC304,37,873
874,QMC5229,6. 0730-2,CTC305,37,874
875,QMC5350,2. 1510-2,EEL010,25,875
876,QMC5350,6. 1010-2,EQA020,16,876


In [16]:
df_lectures['n_lectures'] = df_lectures['Horario'].str.split('-').apply(lambda x: x[-1])
df_lectures

,Codigo,Horario,Sala,Ocupadas,long_lecture,n_lectures
0,ARQ5115,2. 0910-3,ARQ303,27,0,3
1,ARQ5115,2. 1830-4,ARQ202,27,1,4
2,ARQ5115,7. 0730-1,ARQ311,27,2,1
3,ARQ5515,4. 1510-3,ARQ305,24,3,3
4,ARQ5515,4. 1510-3,CTC207,18,4,3
...,...,...,...,...,...,...
873,QMC5229,4. 1010-2,CTC304,37,873,2
874,QMC5229,6. 0730-2,CTC305,37,874,2
875,QMC5350,2. 1510-2,EEL010,25,875,2
876,QMC5350,6. 1010-2,EQA020,16,876,2


In [17]:
df_lectures = df_lectures.loc[df_lectures.index.repeat(df_lectures['n_lectures'])]
df_lectures = df_lectures.reset_index().drop(columns=['index', 'n_lectures'])
df_lectures['lecture'] = df_lectures.index

df_lectures

,Codigo,Horario,Sala,Ocupadas,long_lecture,lecture
0,ARQ5115,2. 0910-3,ARQ303,27,0,0
1,ARQ5115,2. 0910-3,ARQ303,27,0,1
2,ARQ5115,2. 0910-3,ARQ303,27,0,2
3,ARQ5115,2. 1830-4,ARQ202,27,1,3
4,ARQ5115,2. 1830-4,ARQ202,27,1,4
...,...,...,...,...,...,...
1980,QMC5350,2. 1510-2,EEL010,25,875,1980
1981,QMC5350,6. 1010-2,EQA020,16,876,1981
1982,QMC5350,6. 1010-2,EQA020,16,876,1982
1983,QMC5351,6. 1330-2,EQA020,21,877,1983


# Time periods

In [18]:
set([t.split(' ')[-1] for t in np.sort(df_lectures['Horario'].unique())])

{'0730-1',
 '0730-2',
 '0730-3',
 '0820-1',
 '0820-2',
 '0820-3',
 '0820-4',
 '0910-1',
 '0910-2',
 '0910-3',
 '1010-1',
 '1010-2',
 '1100-1',
 '1330-1',
 '1330-2',
 '1330-3',
 '1330-4',
 '1420-2',
 '1420-3',
 '1420-4',
 '1510-2',
 '1510-3',
 '1620-1',
 '1620-2',
 '1710-1',
 '1830-1',
 '1830-2',
 '1830-3',
 '1830-4',
 '2020-2'}

In [19]:
days = np.arange(1,7+1)
times = ['0730', '0820', '0910', '1010', '1100', '1330', '1420', '1510', '1620', '1710', '1830', '1920', '2020', '2110']

time_periods = product(days, times)
time_periods = [str(d)+'-'+t for d, t in time_periods]
time_periods

['1-0730',
 '1-0820',
 '1-0910',
 '1-1010',
 '1-1100',
 '1-1330',
 '1-1420',
 '1-1510',
 '1-1620',
 '1-1710',
 '1-1830',
 '1-1920',
 '1-2020',
 '1-2110',
 '2-0730',
 '2-0820',
 '2-0910',
 '2-1010',
 '2-1100',
 '2-1330',
 '2-1420',
 '2-1510',
 '2-1620',
 '2-1710',
 '2-1830',
 '2-1920',
 '2-2020',
 '2-2110',
 '3-0730',
 '3-0820',
 '3-0910',
 '3-1010',
 '3-1100',
 '3-1330',
 '3-1420',
 '3-1510',
 '3-1620',
 '3-1710',
 '3-1830',
 '3-1920',
 '3-2020',
 '3-2110',
 '4-0730',
 '4-0820',
 '4-0910',
 '4-1010',
 '4-1100',
 '4-1330',
 '4-1420',
 '4-1510',
 '4-1620',
 '4-1710',
 '4-1830',
 '4-1920',
 '4-2020',
 '4-2110',
 '5-0730',
 '5-0820',
 '5-0910',
 '5-1010',
 '5-1100',
 '5-1330',
 '5-1420',
 '5-1510',
 '5-1620',
 '5-1710',
 '5-1830',
 '5-1920',
 '5-2020',
 '5-2110',
 '6-0730',
 '6-0820',
 '6-0910',
 '6-1010',
 '6-1100',
 '6-1330',
 '6-1420',
 '6-1510',
 '6-1620',
 '6-1710',
 '6-1830',
 '6-1920',
 '6-2020',
 '6-2110',
 '7-0730',
 '7-0820',
 '7-0910',
 '7-1010',
 '7-1100',
 '7-1330',
 '7-1420',

In [20]:
df_lectures['Horario'] = df_lectures['Horario'].str.split('-').apply(lambda x: x[0])
df_lectures['Horario'] = df_lectures['Horario'].str.replace('. ', '-')

df_lectures

<ipython-input-20-01be645fcdbb>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df_lectures['Horario'] = df_lectures['Horario'].str.replace('. ', '-')


,Codigo,Horario,Sala,Ocupadas,long_lecture,lecture
0,ARQ5115,2-0910,ARQ303,27,0,0
1,ARQ5115,2-0910,ARQ303,27,0,1
2,ARQ5115,2-0910,ARQ303,27,0,2
3,ARQ5115,2-1830,ARQ202,27,1,3
4,ARQ5115,2-1830,ARQ202,27,1,4
...,...,...,...,...,...,...
1980,QMC5350,2-1510,EEL010,25,875,1980
1981,QMC5350,6-1010,EQA020,16,876,1981
1982,QMC5350,6-1010,EQA020,16,876,1982
1983,QMC5351,6-1330,EQA020,21,877,1983


# Formulation

In [21]:
df_lectures = df_lectures.rename({
    'Codigo': 'course',
    'Horario': 'time_period',
    'Sala': 'room',
    'Ocupadas': 'size',
}, axis='columns')

df_lectures

,course,time_period,room,size,long_lecture,lecture
0,ARQ5115,2-0910,ARQ303,27,0,0
1,ARQ5115,2-0910,ARQ303,27,0,1
2,ARQ5115,2-0910,ARQ303,27,0,2
3,ARQ5115,2-1830,ARQ202,27,1,3
4,ARQ5115,2-1830,ARQ202,27,1,4
...,...,...,...,...,...,...
1980,QMC5350,2-1510,EEL010,25,875,1980
1981,QMC5350,6-1010,EQA020,16,876,1981
1982,QMC5350,6-1010,EQA020,16,876,1982
1983,QMC5351,6-1330,EQA020,21,877,1983


In [22]:
df_rooms = df_rooms.rename({
    'Sala': 'room',
    'Carteiras': 'size'
}, axis='columns')

df_rooms

,Lugar,room,size,Características,Ar condicionado,Projetor
0,ARQ05 - 1ºAndar,ARQ201,45,"2 Ar condicionado, 1 Mesa do professor (sala d...",2,1
1,ARQ04 - 1ºAndar,ARQ202,40,"1 Ar condicionado, 1 Projetor multimídia, 1 Me...",1,1
2,ARQ03 - 1ºAndar,ARQ203,40,"2 Ar condicionado, 1 Projetor multimídia, 1 Me...",2,1
3,ARQ02 - Térreo,ARQ204,40,"2 Ar condicionado, 1 Projetor multimídia, 1 Qu...",2,1
4,ARQ09 - 2º Andar,ARQ301,20,"1 Ar condicionado, 1 Projetor multimídia, 1 Me...",1,1
...,...,...,...,...,...,...
65,NaN,EQA018,45,"50 Carteira com cadeira separada, 1 Projetor m...",2,1
66,NaN,EQA020,35,"2 Ar condicionado, 35 Carteira com cadeira sep...",2,1
67,NaN,EQA021,35,"2 Ar condicionado, 35 Carteira com cadeira sep...",2,1
68,NaN,EQA022,35,"2 Ar condicionado, 35 Carteira com cadeira sep...",2,1


In [81]:
%%time

# basic sets
C = df_lectures['course'].unique()
T = df_lectures['time_period'].unique()
R = df_rooms['room'].unique()
L = df_lectures['lecture'].unique()


# we define the patterns through the `long_lecture`, which groups lectures
# that span through multiple time periods
long_L = df_lectures.groupby('course')['long_lecture'].unique()
longlecture2lectures = df_lectures.groupby('long_lecture')['lecture'].unique()

P = list()
for c in tqdm(C):
    P_c = list(powerset(long_L[c]))[1:]  # remove empty set
    P_c = [[longlecture2lectures[ll] for ll in p] for p in P_c]
    P_c = [np.concatenate(p) for p in P_c]
    P += P_c
P_idx = np.arange(0,len(P))  # so we can hash it

  0%|          | 0/412 [00:00<?, ?it/s]

CPU times: user 2.17 s, sys: 12 ms, total: 2.19 s
Wall time: 2.18 s


In [82]:
# L_c[c] are the lectures of course c
L_c = df_lectures.groupby('course')['lecture'].unique()

# T_l[l] is the time period in which a lecture happens
T_l = {l: df_lectures.iloc[l]['time_period'] for l in L}

In [109]:
# then, P[c] will have all the patterns that preserve contiguous room stability
patterns_course = [df_lectures.loc[p[0]]['course'] for p in P]
P_c = dict()
for c in tqdm(C):
    P_c[c] = [i for i, c_p in enumerate(patterns_course) if c_p == c]

  0%|          | 0/412 [00:00<?, ?it/s]

In [ ]:
# then, P[c] will have all the patterns that preserve contiguous room stability
patterns_course = [df_lectures.loc[p[0]]['course'] for p in P]
P_c = dict()
for c in tqdm(C):
    P_c[c] = [i for i, c_p in enumerate(patterns_course) if c_p == 0]

P_l = dict()
for l in tqdm(L):
    P_l[l] = list()
    for p_i in P_idx:
        if l in P[p_i]:
            P_l[l].append(p_i)

In [ ]:
# seats required by pattern
size = {p_i: df_lectures.loc[P[p_i]]['size'].max() for p_i in tqdm(P_idx)}

# R[p] are the rooms suitable for pattern p
R_p = {p_i: df_rooms[df_rooms['size'] >= size[p_i]]['room'].unique() for p_i in tqdm(P_idx)}

R_c = {c: list({r for p_i in P_c[c] for r in R_p[p_i]}) for c in C}

In [128]:
T_p = {p_i: T_l[l] for p_i in tqdm(P_idx) for l in P[p_i]}

P_rt = dict()
for r in tqdm(R):
    P_rt[r] = dict()
    for t in T:
        P_rt[r][t] = [p_i for p_i in P_idx if r in R_p[p_i] and t in T_p[p_i]]

  0%|          | 0/77148 [00:00<?, ?it/s]

  0%|          | 0/68 [00:00<?, ?it/s]

In [62]:
w = [len(P[p_i]) for p_i in P_idx]

In [146]:
data = {
    'C': C,
    'T': T,
    'R': R,
    'L': L,
    'P': P,
    'L_c': L_c,
    'T_l': T_l,
    'P_c': P_c,
    'P_l': P_l,
    'P_rt': P_rt,
    'R_p': R_p,
    'R_c': R_c,
    'w': w,
}

with open('data.pkl', 'wb') as f:
    pickle.dump(data, f)

In [23]:
with open('data.pkl', 'rb') as f:
    data = pickle.load(f)

C = data['C']
T = data['T']
R = data['R']
L = data['L']
P = data['P']
L_c = data['L_c']
T_l = data['T_l']
P_c = data['P_c']
P_l = data['P_l']
P_rt = data['P_rt']
R_p = data['R_p']
R_c = data['R_c']
w = data['w']

P_idx = np.arange(0, len(P))

# AMPL

In [150]:
from amplpy import AMPL

ampl = AMPL()

ampl.read('classroom.mod')
# ampl.readData('classroom-example.dat')

In [151]:
ampl.getSet('C').setValues(C.astype(str))
ampl.getSet('T').setValues(T.astype(str))
ampl.getSet('R').setValues(R.astype(str))
ampl.getSet('L').setValues(L.astype(str))
ampl.getSet('P').setValues(P_idx.astype(str))

In [152]:
%%time

def set_indexed(ampl, name, var, idx):
    SS = ampl.getSet(name)
    for i in idx:
        try:
            SS[str(i)].setValues(np.array(var[i]).astype(str))
        except RuntimeError as e:
            print(i)
            raise e

set_indexed(ampl, 'L_c', L_c, C)
set_indexed(ampl, 'R_p', R_p, P_idx)
set_indexed(ampl, 'R_c', R_c, C)
set_indexed(ampl, 'P_c', P_c, C)
set_indexed(ampl, 'P_l', P_l, L)

CPU times: user 10.9 s, sys: 772 ms, total: 11.7 s
Wall time: 15.2 s


In [154]:
_P_rt = ampl.getSet('P_rt')
for r in R:
    for t in T:
        _P_rt[str(r),str(t)].setValues(P_rt[r][t])

In [159]:
ampl.exportData('ufsc-instance.dat')